# Deep Dive into Advanced Object-Oriented Programming in Python

---

## 1. Class Construction and Instantiation

- `__new__`: Allocates and returns a new instance before `__init__`
- `__init__`: Initializes instance variables
- `__del__`: Finalizer method—beware of circular references and GC delays
- `__init_subclass__`: Auto-hooks triggered on subclass creation
- `__class_getitem__`: Enables generic class syntax (`Cls[T]`)

---

## 2. Attribute Management and Binding Mechanics

- `__getattr__`: Invoked only for missing attributes
- `__getattribute__`: Intercepts *every* attribute access (handle with care)
- `__setattr__`, `__delattr__`: Manage instance state assignment/deletion
- Attribute Resolution Order:
  - Instance → Class → Inherited Bases (MRO)
  - Fallbacks in `__dict__`, `__class__.__dict__`, etc.
- Shadowing class attributes from instances and vice versa
- Dynamic attribute injection via `setattr()` and introspection

---

## 3. Class-Level Behavior and Namespace Control

- Class attributes shared across instances (unless shadowed)
- Accessing via `ClassName.attr` vs `self.__class__.attr`
- Class namespaces (`__dict__`, `__annotations__`)
- Custom namespace with metaclass `__prepare__`

---

## 4. Method Resolution Order (MRO)

- `mro()` and C3 Linearization
- Diamond inheritance structures
- `super()` and dynamic dispatch
- Classic (Python 2) vs New-Style (Python 3+) differences

---

## 5. Descriptors: Attribute Control at Class Level

- Descriptor Types:
  - Data: defines both `__get__` and `__set__`
  - Non-Data: defines only `__get__`
- `property()` under the hood = data descriptor
- Overriding instance attributes with descriptors
- Class-level caching and validation logic

---

## 6. Encapsulation Techniques

- Public: no underscores
- Protected: `_single_leading_underscore`
- Private/Mangled: `__double_leading_underscore`
- Read-only attributes using `@property` with no setter
- Data hiding via closures or descriptors

---

## 7. Polymorphism: Static and Dynamic

- Duck Typing: "If it quacks like a duck..."
- Runtime Polymorphism via method overriding
- Compile-time mimicry with `Protocol` and `typing` generics
- Operator Overloading via dunder methods
- Liskov Substitution Principle in dynamic systems

---

## 8. Inheritance, Composition, and Delegation

- Inheritance:
  - Single and Multiple
  - Cooperative super calls
- Composition:
  - "Has-a" relationship
  - Delegation patterns using `__getattr__`
- Mixins:
  - One-responsibility-per-class
  - Thin layers for behavior injection

---

## 9. Abstract Base Classes and Interface Emulation

- Using `abc.ABCMeta` or `abc.ABC` base class
- Declaring `@abstractmethod`
- Virtual subclass registration (`MyInterface.register(SomeClass)`)
- Custom `__subclasshook__()` for interface contracts
- Distinguishing interface vs implementation inheritance

---

## 10. Callable and Custom Behaviors

- `__call__`: Make instances behave like functions
- `__contains__`, `__len__`, `__iter__`: Collection interfaces
- `__eq__`, `__hash__`, and immutability constraints
- Emulating numeric types (`__add__`, `__radd__`, `__iadd__`)
- Method chaining with fluent APIs

---

## 11. Memory Efficiency and Object Models

- `__slots__`: Prevents `__dict__`, saves memory
- Tracking memory with `sys.getsizeof()` and `gc.get_stats()`
- Circular references, weak references (`weakref`)
- Class-level caches and singletons (`__new__`)
- Immutable objects and custom `__hash__`, `__eq__`

---

## 12. Metaclass Metaprogramming

- `type` as the metaclass of all new-style classes
- Customizing class construction:
  - `__new__`, `__init__`, `__call__`
- Dynamic class patching at runtime
- Rewriting base classes and MRO via `__mro_entries__`
- Enforcing rules across subclasses (`__init_subclass__`)
- Automatic attribute injection or annotation inspection

---

## 13. Dynamic Class and Method Generation

- Factory functions generating classes dynamically
- Closures to encapsulate behaviors in generated methods
- Decorators creating context-aware class modifications
- Template classes and parameterized type builders
- Evaluating class sources with `exec()` and `type()`

---

## 14. Object Lifecycle and Cloning

- Copying and Deepcopying:
  - `__copy__`, `__deepcopy__`
- Serialization (Pickling):
  - `__reduce__`, `__getstate__`, `__setstate__`
- Lifecycle hooks for caching, reinitialization

---

## 15. Testing and Debugging OOP Code

- Verifying method resolution with `print(Class.__mro__)`
- Mocking instance behaviors
- Method spies and audit decorators
- Injection of test interfaces via monkey patching
- Stress testing attributes with randomized setter validators

---

## 16. Idioms and Patterns Specific to OOP

- Static vs Class Method Use Cases
- Adapter classes using composition
- Strategy Pattern via callable class injection
- Builder Pattern using fluent chained methods
- Observers and signals for decoupled design

---

## 17. Modern Boilerplate Reduction

- `@dataclass` and field management
- Frozen dataclasses as immutable records
- `kw_only`, `slots`, `match_args` (Python 3.10+)
- `attrs` library for richer declarative classes
- Integration with type hints and runtime enforcement

---

## 18. Advanced Properties and Field Descriptors

- Multiple properties sharing a common underlying value
- Validation pipelines using layered properties
- Lazy evaluation using cached properties
- Combining `@property` with `__slots__` and `__setattr__`
- Property injection via metaclass

---

## 19. Class-Level Decorators and Utility Tools

- Decorators modifying class members on definition
- `classmethod` factories for canonical constructors
- Auto-registration of plugin subclasses
- Injecting methods or class-wide configuration at import time
- Encapsulation of configuration using class wrappers

---

## 20. Integration with Type Systems and Protocols

- `Protocol` (from `typing`) for static duck typing
- `runtime_checkable` and dynamic compliance
- `Generic[T]` for typed base classes
- `@overload` for polymorphic interfaces
- `Final`, `Literal`, and `TypeVar` constraints on attributes

---

# Class construction and Initialisation

## The __new__ method

1. __new__: Creating a New Object
What is it?

__new__ is a special method that creates a new instance of your class.

It’s called before __init__.

You don’t usually need to touch this unless you are doing something advanced (e.g., creating immutable objects).

How does it work?

When you do obj = MyClass(), Python:

Calls __new__ to create the object.

Calls __init__ to initialize the object.

__new__ must return the new object.

In [ ]:
class MyClass:
    def __new__(cls):
        print("Creating instance (inside __new__)")
        instance = super().__new__(cls)  # Actually makes the object
        return instance

    def __init__(self):
        print("Initializing instance (inside __init__)")

obj = MyClass()


## 2. __init__: Initializing the Object
What is it?

__init__ sets up your object’s data after it has been created.

This is the method you almost always use when making classes.

How does it work?

You define what data the object should have.

In [ ]:
class Dog:
    def __init__(self, name, age):
        self.name = name   # Save name in the object
        self.age = age     # Save age in the object

my_dog = Dog("Buddy", 5)
print(my_dog.name)  # Buddy
print(my_dog.age)   # 5

# This code demonstrates the use of __new__ and __init__ methods in Python classes.
# __new__ created the Dog object (default behavior).
# __init__ set name and age.

## 3. __del__: Destroying the Object
What is it?

__del__ is called when Python is about to delete the object, often when nothing references it anymore.

Think of it like a cleanup or shutdown step.

### Caution:

You can’t guarantee exactly when this happens.

If there are circular references (two objects referencing each other), it may never be called right away.

In [ ]:
class FileHandler:
    def __del__(self):
        print("Cleaning up FileHandler")

fh = FileHandler()
del fh  # Triggers __del__ (usually immediately)


## 4. __init_subclass__: Reacting to Subclasses

What is it?

This method is called every time you create a new subclass of your class.

You can use it to:

- Automatically enforce rules on subclasses.

- Register subclasses.

- Print messages when a subclass is made.

How does it work?

The method receives the subclass as cls.

In [ ]:
class Animal:
    def __init_subclass__(cls, **kwargs):
        super().__init_subclass__(**kwargs)
        print(f"Animal subclass created: {cls.__name__}")

class Dog(Animal):
    pass

class Cat(Animal):
    pass


## 5. __class_getitem__: Using [] with Classes
What is it?

Lets you do syntax like MyClass[SomeType].

Useful in generics and type annotations.

In [ ]:
class MyContainer:
    def __class_getitem__(cls, item):
        print(f"MyContainer got item: {item}")
        return f"Processed {item}"

result = MyContainer["hello"]
print(result)

# Attribute Management and Binding Mechanics

* This part is about how Python handles attributes:

* When you access them

* When you set or delete them

* How Python looks them up internally

## __getattr__: Handling Missing Attributes

What is it?

A special method called only if an attribute does not exist normally.

How does it work?

You try to access obj.some_attr.

If some_attr is not found, __getattr__ is called.

If some_attr exists, __getattr__ is ignored.

In [ ]:
class LazyAttributes:
    def __getattr__(self, name):
        print(f"Attribute '{name}' was not found. Generating dynamically!")
        return f"Default value for {name}"

obj = LazyAttributes()
print(obj.foo)   # foo does not exist


## __getattribute__: Intercepts All Attribute Access
What is it?

A special method that is called every single time you access any attribute, whether it exists or not.

### Be careful:

If you override this, you must avoid infinite recursion by using super().__getattribute__ to fetch real attributes.

In [ ]:
class Verbose:
    def __getattribute__(self, name):
        print(f"Accessing attribute '{name}'")
        return super().__getattribute__(name)

    def __init__(self):
        self.x = 42

obj = Verbose()
print(obj.x)

## ```__setattr__```: Managing Attribute Assignment
What is it?

Called every time you do ```obj.some_attr = value```.

How does it work?

You have to tell Python how to save the attribute (normally by writing to ```self.__dict__```).

In [ ]:
class Watcher:
    def __setattr__(self, name, value):
        print(f"Setting {name} to {value}")
        super().__setattr__(name, value)

obj = Watcher()
obj.y = 100

## __delattr__: Managing Attribute Deletion
What is it?

Called when you do ```del obj.some_attr```.

In [ ]:
class Deleter:
    def __delattr__(self, name):
        print(f"Deleting {name}")
        super().__delattr__(name)

obj = Deleter()
obj.z = 10
del obj.z

## Attribute Resolution Order (MRO)

How does Python look up attributes when you access obj.name?

Instance Dictionary: ```obj.__dict__```

Class Dictionary: ```obj.__class__.__dict__```

Parent Classes: If the class doesn’t have it, Python searches base classes in method resolution order (MRO).

In plain terms:

1. Python starts at the object.

2. If not found, checks the class.

3. If still not found, checks base classes.

In [ ]:
class A:
    x = "from A"

class B(A):
    pass

b = B()
print(b.x)  # Finds x in A

## Shadowing Class Attributes

What is it?

If you assign to an attribute in the instance, it hides (shadows) the same attribute in the class.

In [ ]:
class MyClass:
    x = "class attribute"

obj = MyClass()
print(obj.x)  # class attribute

obj.x = "instance attribute"
print(obj.x)  # instance attribute

del obj.x
print(obj.x)  # class attribute (falls back)


## Dynamic Attribute Injection via ```setattr()```
What is it?

You can add attributes dynamically with ```setattr()```

Why use this?

1. To create attributes at runtime.

2. To configure objects dynamically.

In [ ]:
class Dynamic:
    pass

d = Dynamic()
setattr(d, "new_attr", 123)
print(d.new_attr)


# 3. Class-Level Behavior and Namespace Control
This topic is about:

How class attributes behave.

How they differ from instance attributes.

What a class namespace is.

Advanced control with metaclasses.

## 1. Class Attributes Shared Across Instances
What is a class attribute?

A variable defined inside a class, outside of any method.

Shared by all instances unless an instance overrides it.

In [ ]:
class Dog:
    species = "Canis familiaris"  # class attribute

    def __init__(self, name):
        self.name = name  # instance attribute

d1 = Dog("Rex")
d2 = Dog("Buddy")

print(d1.species)  # Canis familiaris
print(d2.species)  # Canis familiaris

# Changing the class attribute affects all instances

Dog.species = "Wolf"

print(d1.species)  # Wolf
print(d2.species)  # Wolf

# Changing the instance attribute does not affect the class attribute

d1.species = "Golden Retriever"
print(d1.species)  # Golden Retriever (instance)
print(d2.species)  # Wolf (class)


## Accessing via ```ClassName.attr``` vs ```self.__class__.attr```
How do you get a class attribute?

You can reference it directly with the class name:

In [ ]:
class Cat:
    kind = "feline"

class Lion(Cat):
    pass

l = Lion()
print(l.__class__.kind)  # feline

# ClassName.attr uses a specific class.
# self.__class__.attr dynamically picks the class of the object.

## 3. Class Namespaces (```__dict__, __annotations__```)
What is a namespace?

It’s where Python stores names (variables).

Every class has a ```__dict__```:

This is a mapping of attribute names to their values.

In [ ]:
class Example:
    x = 10
    y = 20

print(Example.__dict__)


In [ ]:
# What about __annotations__?
# If you use type hints, Python stores them here.
# Example:

class Annotated:
    a: int = 5
    b: str = "hello"

print(Annotated.__annotations__)

## 4. Custom Namespace with Metaclass ```__prepare__```
This is an advanced topic—but I’ll explain it simply.

What is a metaclass?

A metaclass is a “class of a class.”

It controls how classes themselves are created.

What does ```__prepare__``` do?

It lets you customize the namespace dictionary used when building the class body.

Why would you do this?

To:

- Track the order of definitions.

- Enforce rules.

- Use a special dictionary-like object.

In [ ]:
class MyMeta(type):
    @classmethod
    def __prepare__(metacls, name, bases):
        print("Preparing namespace for", name)
        return {}  # You could return an OrderedDict here

class MyClass(metaclass=MyMeta):
    x = 1
    y = 2
    z = 3

# 4. Method Resolution Order (MRO)

Method Resolution Order (MRO) is all about:

- How Python figures out which method or attribute to use when you have inheritance, especially multiple inheritance.

- Which parent class gets searched first.

Python 3 uses something called C3 linearisation to make this consistent and predictable.

## 1.  What is mro()?
mro() stands for Method Resolution Order.

Definition:

It’s the order Python uses to look up methods and attributes when you do ```object.method()``` or ```object.attr```.

How to see it:

You can use:

```ClassName.mro()```

or ```ClassName.__mro__```

In [ ]:
class A:
    pass

class B(A):
    pass

class C(B):
    pass

print(C.mro())


## What is C3 Linearization?
C3 Linearization is the algorithm Python uses to compute this order.

Why do we need it?

In multiple inheritance, there can be conflicts or ambiguity about which parent’s method to pick.

The idea of C3 Linearization:

The MRO should:

Be consistent (always same order).

Respect the order of base classes as declared.

Keep subclasses before superclasses.

Preserve the order in which bases are listed.

Key point:

You almost never need to compute C3 yourself, but you do need to understand that Python has a clear, predictable rule to avoid confusion.

## What is the Diamond Problem?

This is why MRO is important.

Diamond inheritance:

Imagine this:

```bash
    A
   / \
  B   C
   \ /
    D
```

Here:

* B inherits from A

* C inherits from A

* D inherits from both B and C

How to read this:

* Look in D.

* Then B.

* Then C.

* Then A.

* Finally object.

In [ ]:
class A:
    def hello(self):
        print("A")

class B(A):
    def hello(self):
        print("B")

class C(A):
    def hello(self):
        print("C")

class D(B, C):
    pass

d = D()
d.hello()

# observe the method resolution order (MRO)
print(D.mro())

## What is super()?
super() is how you call methods from the parent classes according to MRO.
D → B → C → A → object

In [ ]:
class A:
    def greet(self):
        print("Hello from A")

class B(A):
    def greet(self):
        print("Hello from B")
        super().greet()

class C(A):
    def greet(self):
        print("Hello from C")
        super().greet()

class D(B, C):
    def greet(self):
        print("Hello from D")
        super().greet()

d = D()
d.greet()


super() moves to the next in MRO.

So in D:

super().greet() goes to B.greet()

In B, super().greet() goes to C.greet()

In C, super().greet() goes to A.greet()

Tip:

super() is dynamic—it always follows the MRO.

You don’t have to hard-code parent class names.

# 5. Descriptors: Attribute Control at Class Level
What is a descriptor?
A descriptor is any object that implements one or more of these methods:

```__get__(self, instance, owner)```

```__set__(self, instance, value)```

```__delete__(self, instance)```

In plain language:

A descriptor is a special object living in the class, which controls what happens when you do:

```python
obj.attr         # (get)
obj.attr = value # (set)
del obj.attr     # (delete)
```

This is how things like ```@property and @classmethod``` work behind the scenes.

## Descriptor Types
There are two main types of descriptors:

### A) Data Descriptor
Implements both ```__get__()``` and ```__set__()```.

Has higher priority when accessing attributes.

Can override instance attributes.

In [ ]:
class DataDescriptor:
    def __get__(self, instance, owner):
        print("DataDescriptor.__get__ called")
        return 42

    def __set__(self, instance, value):
        print(f"DataDescriptor.__set__ called with {value}")

class MyClass:
    x = DataDescriptor()

obj = MyClass()
print(obj.x)       # Calls __get__
obj.x = 99         # Calls __set__


### B) Non-Data Descriptor
Implements only ```_get__()```.

Lower priority than instance attributes.

If you set an attribute on the instance, it overrides the descriptor.

In [ ]:
class NonDataDescriptor:
    def __get__(self, instance, owner):
        print("NonDataDescriptor.__get__ called")
        return 123

class MyClass:
    y = NonDataDescriptor()

obj = MyClass()
print(obj.y)        # Calls __get__
obj.y = 888         # Overrides descriptor
print(obj.y)        # Returns 888, not __get__

## ```property()``` Is a Data Descriptor
You’ve used property() or @property many times—this is just a built-in data descriptor

In [ ]:
class MyClass:
    def __init__(self):
        self._value = 0

    @property
    def value(self):
        print("getter called")
        return self._value

    @value.setter
    def value(self, v):
        print("setter called")
        self._value = v

obj = MyClass()
obj.value = 10
print(obj.value)


## Overriding Instance Attributes with Descriptors
This is where descriptors are different from regular class variables.

In [ ]:
class Descriptor:
    def __get__(self, instance, owner): # instance is the object, owner is the class
        return "from descriptor" # This is what gets returned when you access the attribute

    def __set__(self, instance, value):
        print("descriptor set called")

class MyClass:
    attr = Descriptor()

obj = MyClass()
obj.attr = "instance value"
print(obj.attr)s

# UP TO CLASS LEVEL CACHING